### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 9, 2)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-09-02
yesterday: 2023-09-01 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 8)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2023-08-24
yesterday: 2023-09-01


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,AIMIRT,2023-08-17,11.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'AIMIRT'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,667,AIMIRT,SET,12.30,13.00,11.70,999.99,1.00,"5,954.81","7,324.41",2.58,0.09,668,2018-06-21 02:41:32.576617,2023-01-31 03:07:59.727836


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "AIMIRT" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,AIMIRT,2023-09-01,10.80,10.90,10.80,"25,701",10.90
1,AIMIRT,2023-08-31,10.90,10.90,10.80,"89,000",10.90
2,AIMIRT,2023-08-30,10.90,11.00,10.80,"220,600",11.00
3,AIMIRT,2023-08-29,10.90,11.00,10.90,"168,905",10.90
4,AIMIRT,2023-08-28,10.90,11.00,10.90,"254,105",10.90


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2023-09-01' AND name IN ('AH', 'AIMIRT', 'AIMIRT', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART')
ORDER BY name, date


29

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2023-08-24' AND name IN ('AH', 'AIMIRT', 'AIMIRT', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART') 
ORDER BY name


29

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(29, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,AH,36.00,34.25,5.11
1,AIMIRT,10.80,10.90,-0.92
2,ASK,25.25,25.75,-1.94
3,ASP,2.94,2.96,-0.68
4,AWC,4.26,4.12,3.40
5,BCH,19.00,19.20,-1.04
6,CPNREIT,11.00,10.80,1.85
7,DIF,9.35,9.15,2.19
8,GVREIT,7.00,7.35,-4.76
9,IVL,29.00,28.75,0.87


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
8,GVREIT,7.00,7.35,-4.76%,Worse
2,ASK,25.25,25.75,-1.94%,Worse
10,JASIF,6.75,6.85,-1.46%,Worse
20,SCC,312.00,316.00,-1.27%,Worse
5,BCH,19.00,19.20,-1.04%,Worse
17,PTG,10.40,10.50,-0.95%,Worse
16,ORI,10.50,10.60,-0.94%,Worse
1,AIMIRT,10.80,10.90,-0.92%,Worse
3,ASP,2.94,2.96,-0.68%,Worse
25,TFFIF,7.50,7.50,0.00%,No Change


In [16]:
file_name = 'trend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(data_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [17]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
1,AIMIRT,10.80,10.90,-0.92,Worse
2,ASK,25.25,25.75,-1.94,Worse
3,ASP,2.94,2.96,-0.68,Worse
5,BCH,19.00,19.20,-1.04,Worse
8,GVREIT,7.00,7.35,-4.76,Worse
10,JASIF,6.75,6.85,-1.46,Worse
16,ORI,10.50,10.60,-0.94,Worse
17,PTG,10.40,10.50,-0.95,Worse
20,SCC,312.00,316.00,-1.27,Worse


In [18]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,65.52%
Worse,31.03%
No Change,3.45%


In [19]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(251, 8)

In [20]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [21]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(29, 13)

In [22]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
20,SCC,316.00,312.00,-1.27%,Worse,SET50
9,IVL,28.75,29.00,0.87%,Better,SET50
18,PTTGC,36.50,37.00,1.37%,Better,SET50
12,JMT,45.50,46.25,1.65%,Better,SET50
4,AWC,4.12,4.26,3.40%,Better,SET50
11,JMART,21.20,23.20,9.43%,Better,SET50


In [23]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,83.33%
Worse,16.67%


In [24]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
5,BCH,19.20,19.00,-1.04%,Worse,SET100
17,PTG,10.50,10.40,-0.95%,Worse,SET100
16,ORI,10.60,10.50,-0.94%,Worse,SET100
19,RCL,21.40,21.50,0.47%,Better,SET100
13,KCE,50.75,51.50,1.48%,Better,SET100
23,STA,13.90,15.30,10.07%,Better,SET100
22,SINGER,10.30,14.10,36.89%,Better,SET100


In [25]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,57.14%
Worse,42.86%


In [26]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
8,GVREIT,7.35,7.00,-4.76%,Worse,SET999
2,ASK,25.75,25.25,-1.94%,Worse,SET999
10,JASIF,6.85,6.75,-1.46%,Worse,SET999
1,AIMIRT,10.90,10.80,-0.92%,Worse,SET999
3,ASP,2.96,2.94,-0.68%,Worse,SET999
25,TFFIF,7.50,7.50,0.00%,No Change,SET999
21,SENA,2.84,2.86,0.70%,Better,SET999
27,WHAIR,6.90,7.00,1.45%,Better,SET999
6,CPNREIT,10.80,11.00,1.85%,Better,SET999
28,WHART,10.40,10.60,1.92%,Better,SET999


In [27]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,62.50%
Worse,31.25%
No Change,6.25%


In [28]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [29]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
